# Power System Datasets Preparation

This script generates the four core datasets used in power system analysis: Buses, Generators, Lines and Loads.

## Buses

In [42]:
import pandas as pd

df_Buses = pd.read_csv('./prerun/buses_GB.csv', comment = "#")
df_Buses.rename(columns={'bus_id': 'name'}, inplace=True)
df_Buses.insert(0, 'BusID', range(1, len(df_Buses) + 1))
df_Buses = df_Buses.loc[:, :'y']
df_Buses = df_Buses.drop(columns=['dc', 'symbol', 'under_construction', 'tags'])

df_Buses

,BusID,name,voltage,x,y
0,1,GB1-220,220,-2.895746,58.256815
1,2,GB10-275,275,-3.719714,56.088133
2,3,GB11-275,275,-3.179692,55.924620
3,4,GB12-275,275,-3.178627,55.903134
4,5,GB13-400,400,-2.938918,55.884694
...,...,...,...,...,...
488,489,way/97551523-400,400,-2.416744,55.948838
489,490,way/97561542-275,275,-1.116723,54.606500
490,491,way/975637991,320,-1.271945,60.293401
491,492,way/98788846-275,275,-3.130160,51.481960


### Asign them into regions

In [43]:
regions = pd.read_csv('./GBPower_raw/gsp_gnode_directconnect_region_lookup.csv')
regions

,ng_id,ggd_id,gnode_id,gnode_name,gnode_lat,gnode_lon,gsp_id,gsp_name,gsp_lat,gsp_lon,dc_id,dc_name,dc_lat,dc_lon,region_id,region_name,has_pv,pes_id,pes_name
0,1,1,1.0,COWL,51.71105,-1.18901,1.0,COWL_1,51.711050,-1.189010,NaN,NaN,NaN,NaN,1.0,Cowley,1,20.0,_H
1,2,2,2.0,ECLA,51.92579,-0.90565,2.0,ECLA_1,51.925790,-0.905650,NaN,NaN,NaN,NaN,19.0,East Claydon,1,11.0,_B
2,3,3,3.0,ENDEDW,52.59726,-1.21548,3.0,ENDE_1,52.597260,-1.215480,NaN,NaN,NaN,NaN,20.0,Enderby,1,11.0,_B
3,4,4,4.0,GRENDW,52.24269,-0.72736,4.0,GREN_1,52.242690,-0.727360,NaN,NaN,NaN,NaN,21.0,Grendon,1,11.0,_B
4,5,5,5.0,RATS,52.86342,-1.25444,5.0,RATS_1,52.863420,-1.254440,NaN,NaN,NaN,NaN,22.0,Ratcliffe,1,11.0,_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,376,376,360.0,BPGR,56.01141,-3.72183,NaN,NaN,NaN,NaN,17.0,T_BPGRD-2,56.01141,-3.72183,NaN,NaN,0,NaN,NaN
376,377,377,NaN,NaN,NaN,NaN,335.0,KNAR,54.031211,-1.487844,NaN,NaN,NaN,NaN,132.0,Knaresborough,1,15.0,_F
377,378,378,NaN,NaN,NaN,NaN,336.0,WYLF_1,53.413644,-4.482279,NaN,NaN,NaN,NaN,114.0,Wylfa,1,13.0,_D
378,379,379,NaN,NaN,NaN,NaN,337.0,DALM3,55.838567,-4.208346,NaN,NaN,NaN,NaN,260.0,Dalmarnock,1,18.0,_N


### Load the data

In [44]:
import geopandas as gpd
from shapely.geometry import Point

gdf_regions = gpd.read_file("./GBPower_raw/gsp_geometries.geojson")
gdf_regions = gdf_regions.to_crs("EPSG:4326")
gdf_buses = gpd.GeoDataFrame(
    df_Buses,
    geometry=gpd.points_from_xy(df_Buses['x'], df_Buses['y']),
    crs="EPSG:4326"
)

gdf_regions = gdf_regions[gdf_regions.is_valid]

### Match Buses into regions

In [45]:
# Match each bus to the region polygon it falls within
gdf_joined = gpd.sjoin(gdf_buses, gdf_regions, how="left", predicate="within")

cols_to_keep = ['BusID', 'name', 'x', 'y', 'RegionName']
gdf_joined[cols_to_keep]

,BusID,name,x,y,RegionName
0,1,GB1-220,-2.895746,58.256815,NaN
1,2,GB10-275,-3.719714,56.088133,Devonside
2,3,GB11-275,-3.179692,55.924620,Whitehouse
3,4,GB12-275,-3.178627,55.903134,Kaimes
4,5,GB13-400,-2.938918,55.884694,Cockenzie
...,...,...,...,...,...
488,489,way/97551523-400,-2.416744,55.948838,Dunbar
489,490,way/97561542-275,-1.116723,54.606500,Lackenby
490,491,way/975637991,-1.271945,60.293401,Fraserburgh
491,492,way/98788846-275,-3.130160,51.481960,Cardiff East


In [46]:
df_Buses['RegionName'] = gdf_joined['RegionName']
df_Buses

,BusID,name,voltage,x,y,RegionName
0,1,GB1-220,220,-2.895746,58.256815,NaN
1,2,GB10-275,275,-3.719714,56.088133,Devonside
2,3,GB11-275,275,-3.179692,55.924620,Whitehouse
3,4,GB12-275,275,-3.178627,55.903134,Kaimes
4,5,GB13-400,400,-2.938918,55.884694,Cockenzie
...,...,...,...,...,...,...
488,489,way/97551523-400,400,-2.416744,55.948838,Dunbar
489,490,way/97561542-275,275,-1.116723,54.606500,Lackenby
490,491,way/975637991,320,-1.271945,60.293401,Fraserburgh
491,492,way/98788846-275,275,-3.130160,51.481960,Cardiff East


### Asign them into different colors

In [47]:
Buses_combined = pd.read_csv('./prerun/buses_mapping/buses_combined.csv',dtype = {'bus385_id': str})
bus_501_385_mapping = dict(zip(Buses_combined['bus501_id'], Buses_combined['bus385_id']))
bus_501_385_mapping

{'way/975637991': nan,
 'relation/8087295-275': '6441',
 'way/495624575-275': '6442',
 'way/567187951-275': nan,
 'way/1262971878': '6443',
 'way/1262971878-275': nan,
 'GB1-220': nan,
 'GB2-220': nan,
 'GB3-220': nan,
 'GB4-220': nan,
 'GB5-220': nan,
 'way/232319495-275': nan,
 'way/624256368-275': nan,
 'way/630939126-275': nan,
 'GB6-220': nan,
 'way/306451400-275': nan,
 'way/115277975-275': nan,
 'way/166703766-400': nan,
 'way/166703766-220': nan,
 'way/166703766-275': nan,
 'way/568527671': '6447',
 'way/778717371-275': nan,
 'way/1185489725-400': nan,
 'way/1185489725-220': '6448',
 'way/1190395478-400': nan,
 'way/1190395478-220': '8013',
 'way/133323988-400': nan,
 'way/133323988-275': '6485',
 'way/68515955-400': '6438',
 'way/68515955-275': nan,
 'way/842685465-275': nan,
 'way/820229174-275': '6440',
 'way/820237569-400': nan,
 'way/239248206-400': '6437',
 'way/689624522-275': nan,
 'way/762272690-275': nan,
 'way/127045977-275': nan,
 'way/118821349-400': nan,
 'way/118

In [48]:
df_Buses['color'] = df_Buses['name'].map(bus_501_385_mapping).apply(
    lambda x: 'green' if pd.notna(x) else 'red'
)
df_Buses.rename(columns={'name': 'Bus name'}, inplace=True)
df_Buses.to_csv('Buses.csv', index=False)

## Loads

In [49]:
df_Loads = pd.read_csv('./GBPower_raw/load_weights.csv')
df_Loads.insert(0, 'LoadID', range(1, len(df_Loads) + 1))
df_Loads

,LoadID,name,load_weight
0,1,8838,0.002569
1,2,8013,0.000261
2,3,8520,0.000241
3,4,8649,0.004041
4,5,4950,0.003187
...,...,...,...
380,381,7458,0.001419
381,382,8283,0.005545
382,383,8094,0.004038
383,384,8093,0.004038


In [50]:
df_bus385_to_501 = pd.read_csv('./prerun/bus385_to_501.csv', dtype={'bus385_id': str})
df_bus385_to_501

,bus385_id,bus501_id
0,6441,relation/8087295-275
1,6442,way/495624575-275
2,6443,way/1262971878
3,6447,way/568527671
4,6449,way/1185489725-220
...,...,...
380,8016,way/26938722-400
381,5210,way/26938722-400
382,5208,way/34656007-400
383,5207,relation/9572254-400


In [51]:
map_385_to_501 = dict(zip(df_bus385_to_501['bus385_id'], df_bus385_to_501['bus501_id']))

df_Loads['name'] = df_Loads['name'].astype(str)
df_Loads['Bus name'] = df_Loads['name'].map(map_385_to_501)

df_Loads['name'] = df_Loads['Bus name']

df_Loads.drop(columns=['Bus name'], inplace=True)
df_Loads.rename(columns={'name': 'Bus name'}, inplace=True)
df_Loads

,LoadID,Bus name,load_weight
0,1,way/92419253-275,0.002569
1,2,way/1190395478-220,0.000261
2,3,way/262523325-400,0.000241
3,4,way/49499923-400,0.004041
4,5,way/25935453-400,0.003187
...,...,...,...
380,381,GB71-275,0.001419
381,382,GB52-400,0.005545
382,383,way/87464485-400,0.004038
383,384,way/87464485-400,0.004038


In [52]:
Bus_region_map = dict(zip(df_Buses['Bus name'], df_Buses['RegionName']))
df_Loads['RegionName'] = df_Loads['Bus name'].map(Bus_region_map)
df_Loads.to_csv('Loads.csv', index=False)

## Generators

In [53]:
df_Units = pd.read_csv('./prerun/units.csv')
df_Units.insert(0, 'UnitID', range(1, len(df_Units) + 1))
df_Units.rename(columns={'NationalGridBmUnit': 'name','carrier': 'type', 'bus': 'Bus name' ,'lon': 'x', 'lat': 'y'}, inplace=True)
df_Units['Bus name'] = df_Units['Bus name'].map(map_385_to_501)
df_Units.to_csv('Units.csv', index=False)
df_Units

,UnitID,name,type,Bus name,x,y,capacity
0,1,IBG-EGEN1,interconnector,way/814351697-400,0.6530000000000001,51.46000000000001,0.0
1,2,IBG-ENGI1,interconnector,way/814351697-400,0.6530000000000001,51.46000000000001,0.0
2,3,IBG-RSQ1,interconnector,way/814351697-400,0.6530000000000001,51.46000000000001,0.0
3,4,IVD-VKL1,interconnector,way/895564309-400,-0.221,52.931,0.0
4,5,IVG-VKL1,interconnector,way/895564309-400,-0.221,52.931,0.0
...,...,...,...,...,...,...,...
1060,1061,SFERD-1,demand_flex,NaN,distributed,distributed,240.0
1061,1062,EAS-SEL01,demand_flex,NaN,distributed,distributed,0.0
1062,1063,SEE-PGU01,demand_flex,NaN,distributed,distributed,0.0
1063,1064,YEP-ATL01,demand_flex,NaN,distributed,distributed,0.0


In [54]:
df_Units_Tech_mapping = pd.read_csv('Units_Tech_mapping.csv')
df_Units_Tech_mapping

,Unit,Technology,cost(EUR/MWh),Unnamed: 3
0,battery,battery storage,0.000000,NaN
1,biomass,biomass,14.893617,NaN
2,cascade,hydro,0.000000,NaN
3,demand_flex,NaN,0.000000,NaN
4,fossil,gas,20.100000,NaN
5,hydro,hydro,0.000000,NaN
6,interconnector,NaN,0.000000,NaN
7,load,NaN,0.000000,NaN
8,nuclear,nuclear,11.380000,NaN
9,offwind,offwind,0.020000,NaN


In [55]:
Units_Tech_mapping = dict(zip(df_Units_Tech_mapping['Unit'], df_Units_Tech_mapping['Technology']))
df_Units['Technology'] = df_Units['type'].map(Units_Tech_mapping)

Tech_Cost_mapping = dict(zip(df_Units_Tech_mapping['Technology'], df_Units_Tech_mapping['cost(EUR/MWh)']))
df_Units['Cost'] = df_Units['Technology'].map(Tech_Cost_mapping)
df_Units.to_csv('Units.csv', index=False)

## Lines

In [56]:
df_Lines = pd.read_csv('./prerun/lines_GB.csv')
df_Lines.insert(0, 'LineID', range(1, len(df_Lines) + 1))
cols_to_keep = ['LineID', 'bus0', 'bus1', 'voltage', 's_nom', 'r', 'x', 'b', 'length', 'under_construction']
df_Lines = df_Lines[cols_to_keep]
df_Lines.to_csv('Lines.csv', index=False)
df_Lines

,LineID,bus0,bus1,voltage,s_nom,r,x,b,length,under_construction
0,1,way/1307462736-275,way/86071326-275,275,921.668,2.200334,14.577213,0.000228,55008.35,f
1,2,relation/7881280-400,relation/13110369-400,400,3574.953,0.420393,3.447219,0.000243,28026.17,f
2,3,way/101077623-275,GB41-275,275,1843.335,0.046871,0.310518,0.000019,2343.53,f
3,4,way/95279334-400,way/103915328-400,400,3574.953,0.425859,3.492045,0.000246,28390.61,f
4,5,way/111528318-220,GB8-220,220,1474.668,1.584632,7.949573,0.000933,79231.62,f
...,...,...,...,...,...,...,...,...,...,...
585,586,relation/13110369-400,way/314338790-400,400,3574.953,0.103282,0.846912,0.000060,6885.46,f
586,587,relation/7879831-400,way/95749833-400,400,3574.953,0.929985,7.625876,0.000538,61998.99,f
587,588,way/96067520-275,GB59-275,275,1843.335,0.214388,1.420323,0.000089,10719.42,f
588,589,GB61-275,way/96067520-275,275,1843.335,0.173507,1.149483,0.000072,8675.34,f


## Links

In [57]:
df_Links = pd.read_csv('./prerun/links_GB.csv')
df_Links.insert(0, 'LinkID', range(1, len(df_Links) + 1))
df_Links.rename(columns={'link_id': 'name'}, inplace=True)
cols_to_keep = ['LinkID', 'name', 'bus0', 'bus1', 'voltage', 'p_nom', 'length', 'under_construction']
df_Links = df_Links[cols_to_keep]
df_Links.to_csv('Links.csv', index=False)
df_Links

,LinkID,name,bus0,bus1,voltage,p_nom,length,under_construction
0,1,relation/10377412-320-DC,way/753113423,way/636630368-400,320,1000,230137.18,f
1,2,relation/13295785-515-DC,relation/12832053,way/642490160,515,1400,723362.06,f
2,3,relation/14126301-450-DC,way/109189896,way/920127890,450,1000,253393.42,f
3,4,relation/15772117-320-DC,way/775577829-400,way/775577827,320,1000,51576.89,f
4,5,relation/15775538-600-DC,way/297840585,way/375626500-400,600,2250,397643.50,f
5,6,relation/15777152-320-DC,way/1262971878,way/975637991,320,600,282108.95,f
6,7,relation/15781671-525-DC,way/1078532568-400,way/1101832649,525,1400,740934.57,f
7,8,relation/2127794-270-DC,way/23281217-400,way/1271288346,270,2000,69019.25,f
8,9,relation/6914309-500-DC,way/137050527-275,way/148382435,500,500,61383.39,f
9,10,relation/8099179-320-DC,way/1262971878,way/568527671,320,1200,161431.49,f


### Add buses outside UK

In [58]:
Buses_EU = pd.read_csv('./Pypsa-eur_raw/buses.csv')
linked_buses = pd.Series(
    pd.concat([df_Links['bus0'], df_Links['bus1']]).unique()
)

Buses_linked = Buses_EU[Buses_EU['bus_id'].isin(linked_buses)]
Buses_linked

,bus_id,voltage,dc,symbol,under_construction,tags,x,y,country,geometry
1823,relation/10695345-400,400,f,Substation,f,relation/10695345,1.345639,51.309878,GB,POINT (1.3456388979917684 51.30987814424089)
1847,relation/12832053,515,t,Substation,f,relation/12832053,6.654777,59.530101,NO,POINT (6.654776609920736 59.530100593739064)
2291,way/1078532568-400,400,f,Substation,f,way/1078532568,-0.236635,52.920885,GB,POINT (-0.2366345394689085 52.92088491851833)
2322,way/109189896,450,t,Substation,f,way/109189896,0.716158,51.440498,GB,POINT (0.7161575436002887 51.44049829914561)
2353,way/1101832649,525,t,Substation,f,way/1101832649,9.167977,55.505082,DK,POINT (9.1679769052874 55.505082071601)
2733,way/1262971878,320,t,Substation,f,way/1262971878,-3.452185,58.479968,GB,POINT (-3.452184943694071 58.47996790376149)
2762,way/1271288346,270,t,Substation,f,way/1271288346,1.782003,50.903459,FR,POINT (1.7820027522149031 50.90345900241787)
2947,way/137050527-275,275,f,Substation,f,way/137050527,-5.770344,54.843315,GB,POINT (-5.7703440335367775 54.84331485406556)
3309,way/148382435,500,t,Substation,f,way/148382435,-4.980751,55.069699,GB,POINT (-4.9807507247075335 55.069699109091)
4237,way/23281217-400,400,f,Substation,f,way/23281217,0.975236,51.105943,GB,POINT (0.9752363053510982 51.1059427193376)


In [59]:
non_gb_buses = Buses_linked[Buses_linked["country"] != "GB"]

new_buses = non_gb_buses.rename(columns={
    "bus_id": "Bus name",
    "x": "x",
    "y": "y"
}).copy()

new_buses["RegionName"] = "Overseas"  # or set based on logic
new_buses["color"] = "yellow"
new_buses["BusID"] = range(df_Buses["BusID"].max() + 1, df_Buses["BusID"].max() + 1 + len(new_buses))

new_buses = new_buses[["BusID", "Bus name", "x", "y", "RegionName", "color"]]

df_Buses = pd.concat([df_Buses, new_buses], ignore_index=True)
df_Buses = df_Buses.drop_duplicates(subset=["Bus name"])
df_Buses.to_csv('Buses.csv', index=False)

## Transformers

In [60]:
df_Transformers = pd.read_csv('./prerun/transformers_GB_modified.csv', header = None)
# Set the first row as header
df_Transformers.columns = df_Transformers.iloc[0]

# Drop the first row now that it’s used as header
df_Transformers = df_Transformers.drop(index=0).reset_index(drop=True)

df_Transformers.rename(columns={'transformer_id': 'TransformerID'}, inplace=True)
cols_to_keep = ['TransformerID', 'bus0', 'bus1', 'voltage_bus0', 'voltage_bus1', 's_nom']
df_Transformers = df_Transformers[cols_to_keep]
df_Transformers.to_csv('Transformers.csv', index=False)

df_Transformers

,TransformerID,bus0,bus1,voltage_bus0,voltage_bus1,s_nom
0,0,GB74-275,GB74-400,275,400,3575
1,1,relation/13110368-275,relation/13110368-400,275,400,7150
2,2,relation/14921826-275,relation/14921826-400,275,400,3575
3,3,relation/7879831-275,relation/7879831-400,275,400,7150
4,4,relation/7880111-275,relation/7880111-400,275,400,2766
...,...,...,...,...,...,...
130,NaN,way/536905500-400,way/536905500,400.0,400,NaN
131,NaN,way/568527671,way/166703766-275,320.0,275,NaN
132,NaN,way/636630368-400,way/636630368,400.0,320,NaN
133,NaN,way/642490160,way/310502642-275,515.0,275,NaN
